In [13]:
#import libraries and classes
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Binarizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [2]:
df_digits = pd.read_csv('../mnist.csv')
df_digits.head()

X = df_digits[df_digits.columns[:-1]].to_numpy()
y = df_digits[df_digits.columns[-1]].to_numpy()

train = 5000
val = 60000


#Split the data as in Assignment #3
X_train, X_val, X_test = X[:train], X[train:val], X[val:]
y_train, y_val, y_test = y[:train], y[train:val], y[val:]

In [3]:
%%time
#going to try to improve my SVC classifier from Assignment #3 with bagging
#the SVC classifier performed best with the MNIST data reduced to 150 features.

preprocess_pipeline = Pipeline([('StandardScaler', StandardScaler()), ('Binarizer', Binarizer())])


preprocess_pipeline_reduced = Pipeline([('StandardScaler', StandardScaler()), ('Binarizer', Binarizer()),
                                        ('Principle Compnent Analysis', PCA(n_components=150))])

X_train_proc = preprocess_pipeline.fit_transform(X_train)

X_train_proc_reduced = preprocess_pipeline_reduced.fit_transform(X_train)


CPU times: total: 1.45 s
Wall time: 649 ms


In [4]:
#SVC classifier using optimized paramters from Assignment #3 plus probability set True so the bagging 
#classifier can use soft voting.

svm_clf = SVC(gamma='scale', kernel='poly', probability=True, random_state=42)

In [ ]:
#Generate a baseline set of accuracy scores
print('one SVC', cross_val_score(svm_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#one SVC [0.98025  0.978375 0.98     0.97725  0.98    ]
#CPU times: total: 13.7 s
#Wall time: 11min 55s

In [ ]:
%%time
#create BaggingClassifier
bag_clf = BaggingClassifier(svm_clf, n_estimators=50, max_samples=1000, random_state=42, n_jobs=-1)

print('bagging SVC', cross_val_score(bag_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#bagging SVC [0.94725  0.939875 0.9465   0.9385   0.945   ]
#CPU times: total: 93.8 ms
#Wall time: 2min 19s



In [ ]:
%%time
#increase max samples.  Better, but 12 times the time for 5 times the samples.  Still not as good 
#as just one pass on the full training set.  What about increasing the estimators by 10x while
#keeping the samples small.

bag_clf = BaggingClassifier(svm_clf, n_estimators=50, max_samples=5000, random_state=42, n_jobs=-1)

print('bagging SVC', cross_val_score(bag_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#bagging SVC [0.967625 0.95975  0.967375 0.9605   0.964   ]
#CPU times: total: 109 ms
#Wall time: 25min 20s

In [ ]:
%%time

bag_clf = BaggingClassifier(svm_clf, n_estimators=500, max_samples=1000, random_state=42, n_jobs=-1)

print('bagging SVC', cross_val_score(bag_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#bagging SVC [0.948125 0.939    0.947125 0.937    0.944875]
#CPU times: total: 125 ms
#Wall time: 22min 21s


In [ ]:
%%time

#try boosting.  Check the time on 3 estimators.  It seems it should be n_estimators * the basline 11 
#minutes above.

#so worse.  Overfitting?  Try fitting to the training set and predicting on that.

ada_clf = AdaBoostClassifier(svm_clf, n_estimators=3, learning_rate=1.0, random_state=42)

print('AdaBoosting SVC', cross_val_score(ada_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

In [ ]:
%%time
###remember that you didn't do much in the way of data reduction optimization in Assignment #3

ada_clf.fit(X_train_proc_reduced, y_train)
y_hat = ada_clf.predict(X_train_proc_reduced)
print('accuracy', accuracy_score(y_train, y_hat))

In [5]:
%%time
#Generate a baseline set of accuracy scores using 5000 instances in the training set
print('one SVC', cross_val_score(svm_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#one SVC [0.945 0.95  0.956 0.963 0.956]
#CPU times: total: 93.8 ms
#Wall time: 13.7 s


one SVC [0.945 0.95  0.956 0.963 0.956]
CPU times: total: 93.8 ms
Wall time: 13.7 s


In [6]:
%%time
#AdaBoost with 5000 instances of training set
ada_clf = AdaBoostClassifier(svm_clf, n_estimators=3, learning_rate=1.0, random_state=42)

print('AdaBoosting SVC', cross_val_score(ada_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#AdaBoosting SVC [0.85  0.861 0.863 0.874 0.836]
#CPU times: total: 62.5 ms
#Wall time: 1min 27s

AdaBoosting SVC [0.85  0.861 0.863 0.874 0.836]
CPU times: total: 62.5 ms
Wall time: 1min 27s


In [7]:
%%time
#AdaBoost with 5000 instances of training set
ada_clf = AdaBoostClassifier(svm_clf, n_estimators=10, learning_rate=0.5, random_state=42)

print('AdaBoosting SVC', cross_val_score(ada_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#AdaBoosting SVC [0.728 0.71  0.79  0.522 0.53 ]
#CPU times: total: 15.6 ms
#Wall time: 5min 27s

AdaBoosting SVC [0.728 0.71  0.79  0.522 0.53 ]
CPU times: total: 15.6 ms
Wall time: 5min 27s


In [10]:
%%time
#try AdaBoost with a Decision stump like in the book
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=50, learning_rate=0.5, 
                            random_state=42)

print('AdaBoosting DTS', cross_val_score(ada_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))
#AdaBoosting DTS [0.658 0.663 0.628 0.675 0.519]
#CPU times: total: 93.8 ms
#Wall time: 16.1 s

AdaBoosting DTS [0.658 0.663 0.628 0.675 0.519]
CPU times: total: 93.8 ms
Wall time: 16.1 s


In [12]:
%%time
#try Decision stump alone
print('DTS', cross_val_score(DecisionTreeClassifier(max_depth=1), X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#DTS [0.2   0.201 0.202 0.211 0.206]
#CPU times: total: 15.6 ms
#Wall time: 428 ms

DTS [0.2   0.201 0.202 0.211 0.206]
CPU times: total: 15.6 ms
Wall time: 428 ms


In [13]:
%%time
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=500, learning_rate=0.5, 
                            random_state=42)

print('AdaBoosting DTS', cross_val_score(ada_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#AdaBoosting DTS [0.62  0.497 0.648 0.616 0.474]
#CPU times: total: 31.2 ms
#Wall time: 3min 11s

AdaBoosting DTS [0.62  0.497 0.648 0.616 0.474]
CPU times: total: 31.2 ms
Wall time: 3min 11s


In [14]:
%%time
#so is the stump because it the book example only had two classes? A max depth of 4 accomodates up 
#to 16 classes and there are 10 digits
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4), n_estimators=50, learning_rate=0.5, 
                            random_state=42)

print('AdaBoosting DTS', cross_val_score(ada_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#AdaBoosting DTS [0.674 0.673 0.705 0.74  0.698]
#CPU times: total: 78.1 ms
#Wall time: 1min 7s

AdaBoosting DTS [0.674 0.673 0.705 0.74  0.698]
CPU times: total: 78.1 ms
Wall time: 1min 7s


In [15]:
#how about a 16 leaf decision tree baseline

print('DTS', cross_val_score(DecisionTreeClassifier(max_depth=4), X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#DTS [0.569 0.569 0.579 0.583 0.575]

DTS [0.569 0.569 0.579 0.583 0.575]


In [16]:
#lower the learning rate
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4), n_estimators=50, learning_rate=0.1, 
                            random_state=42)

print('AdaBoosting DTS', cross_val_score(ada_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

AdaBoosting DTS [0.756 0.719 0.773 0.796 0.753]


In [18]:
%%time
#AdaBoost with 5000 instances of training set.  Perhaps my learning rates have been too high.
ada_clf = AdaBoostClassifier(svm_clf, n_estimators=10, learning_rate=0.01, random_state=42)

print('AdaBoosting SVC', cross_val_score(ada_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#learning rate = 0.1
#AdaBoosting SVC [0.875 0.803 0.825 0.886 0.795]
#CPU times: total: 31.2 ms
#Wall time: 5min 32s

#learning rate = 0.01
#AdaBoosting SVC [0.794 0.777 0.813 0.815 0.825]
#CPU times: total: 46.9 ms
#Wall time: 6min 20s


AdaBoosting SVC [0.794 0.777 0.813 0.815 0.825]
CPU times: total: 46.9 ms
Wall time: 6min 20s


In [19]:
%%time
#grid search for DecisionTree with depth of 4

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4), random_state=42)

param_grid = [{'n_estimators':[10, 50, 100, 500], 'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1]}]

gs = GridSearchCV(ada_clf, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
gs.fit(X_train_proc_reduced, y_train)
print('best params', gs.best_params_)

#best params {'learning_rate': 1, 'n_estimators': 500}
#CPU times: total: 8min 26s
#Wall time: 1h 24min 26s


best params {'learning_rate': 1, 'n_estimators': 500}
CPU times: total: 8min 26s
Wall time: 1h 24min 26s


In [20]:
%%time

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4), n_estimators=500, learning_rate=1, 
                            random_state=42)

print('AdaBoosting DTS', cross_val_score(ada_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#AdaBoosting DTS [0.844 0.862 0.841 0.88  0.835]
#CPU times: total: 46.9 ms
#Wall time: 12min 32s


AdaBoosting DTS [0.844 0.862 0.841 0.88  0.835]
CPU times: total: 46.9 ms
Wall time: 12min 32s


In [21]:
%%time

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4), n_estimators=500, learning_rate=2, 
                            random_state=42)

print('AdaBoosting DTS', cross_val_score(ada_clf, X_train_proc_reduced, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#AdaBoosting DTS [0.86  0.86  0.853 0.865 0.837]
#CPU times: total: 62.5 ms
#Wall time: 13min 2s

AdaBoosting DTS [0.86  0.86  0.853 0.865 0.837]
CPU times: total: 62.5 ms
Wall time: 13min 2s


In [6]:
##was using reduced features above, but the RandomForestModel performed better with 
#the full set of features in Assignment #3.  Try with the full set of features.

#better and much faster.  Is the much faster because I just booted this machine up?

%%time

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4), n_estimators=500, learning_rate=2, 
                            random_state=42)

print('AdaBoosting DTS full features', cross_val_score(ada_clf, X_train_proc, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#AdaBoosting DTS full features [0.907 0.894 0.912 0.923 0.897]
#CPU times: total: 78.1 ms
#Wall time: 2min 37s


AdaBoosting DTS full features [0.907 0.894 0.912 0.923 0.897]
CPU times: total: 78.1 ms
Wall time: 2min 37s


In [8]:
%%time
#playing a bit with tree depth.  Adding 1 to depth should double the splits and double the time.

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=500, learning_rate=2, 
                            random_state=42)

print('AdaBoosting DTS full features', cross_val_score(ada_clf, X_train_proc, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#AdaBoosting DTS full features [0.911 0.906 0.916 0.93  0.921]
#CPU times: total: 78.1 ms
#Wall time: 3min 9s

AdaBoosting DTS full features [0.911 0.906 0.916 0.93  0.921]
CPU times: total: 78.1 ms
Wall time: 3min 9s


In [4]:
%%time
#playing a bit with tree depth.  Adding 1 to depth (6) while I try to look up how to do this 
#with grid search

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=6), n_estimators=500, learning_rate=2, 
                            random_state=42)

print('AdaBoosting DTS full features depth 6', cross_val_score(ada_clf, X_train_proc, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#AdaBoosting DTS full features depth 6 [0.93  0.921 0.932 0.934 0.925]
#CPU times: total: 46.9 ms
#Wall time: 3min 41s

AdaBoosting DTS full features depth 6 [0.93  0.921 0.932 0.934 0.925]
CPU times: total: 46.9 ms
Wall time: 3min 41s


In [5]:
%%time
#playing a bit with tree depth.  Adding 1 to depth (7) while I try to look up how to do this 
#with grid search

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=7), n_estimators=500, learning_rate=2, 
                            random_state=42)

print('AdaBoosting DTS full features depth 7', cross_val_score(ada_clf, X_train_proc, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#AdaBoosting DTS full features depth 7 [0.936 0.931 0.93  0.942 0.941]
#CPU times: total: 46.9 ms
#Wall time: 4min 4s

AdaBoosting DTS full features depth 7 [0.936 0.931 0.93  0.942 0.941]
CPU times: total: 46.9 ms
Wall time: 4min 4s


In [ ]:
##I think I figured it out.  Go go stack overflow.
param_grid = {'base_estimator__max_depth':[7, 8, 9, 10]}

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=500, learning_rate=2, 
                            random_state=42)

gs = GridSearchCV(ada_clf, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
gs.fit(X_train_proc, y_train)
print('best params', gs.best_params_)

In [ ]:
### about three hours of work lost here because save stopped working.  Here's the best classifier that 
#came out of that.  Training it on the full set might hit the mark.  It was really close in 
#earlier testing.

%%time
###double the estimators
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=9), n_estimators=1000, 
                             learning_rate = 3, random_state=42)

print('AdaBoosting DTS', cross_val_score(ada_clf, X_train_proc, y_train, cv=5, scoring='accuracy', n_jobs=-1))
#AdaBoosting DTS [0.944 0.943 0.942 0.95  0.942]
#CPU times: total: 46.9 ms
#Wall time: 10min 39s


In [4]:
%%time
#apparently this thing comes with it's own estimator.  It mentions regression trees.
#not bad for a start.  

gb_clf = GradientBoostingClassifier(max_depth=9)

print('GBC', cross_val_score(gb_clf, X_train_proc, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#GBC [0.915 0.911 0.922 0.919 0.901]

GBC [0.915 0.911 0.922 0.919 0.901]


In [6]:
%%time
hgb_clf = HistGradientBoostingClassifier(max_depth=9, random_state=42)

print('GBC', cross_val_score(hgb_clf, X_train_proc, y_train, cv=5, scoring='accuracy', n_jobs=-1))

#GBC [0.945 0.928 0.939 0.94  0.944]
#pretty nice and fast too

GBC [0.945 0.928 0.939 0.94  0.944]


In [7]:
%%time
hgb_clf = HistGradientBoostingClassifier(max_depth=9, random_state=42)

param_grid = [{'max_iter':[10, 50, 100, 500], 'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1]}]

gs = GridSearchCV(hgb_clf, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
gs.fit(X_train_proc, y_train)
print('best params', gs.best_params_)
print('best score', gs.best_score_)

#best params {'learning_rate': 0.1, 'max_iter': 500}
#best score 0.9400000000000001
#CPU times: total: 11min 33s
#Wall time: 1h 47min 29s


best params {'learning_rate': 0.1, 'max_iter': 500}
CPU times: total: 11min 33s
Wall time: 1h 47min 29s


In [9]:
%%time
hgb_clf = HistGradientBoostingClassifier(max_depth=9, random_state=42)

param_grid = [{'max_iter':[250, 500, 1000], 'learning_rate':[0.05, 0.1, 0.2, 0.4]}]

gs = GridSearchCV(hgb_clf, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
gs.fit(X_train_proc, y_train)
print('best params', gs.best_params_)
print('best score', gs.best_score_)
#best params {'learning_rate': 0.05, 'max_iter': 500}
#best score 0.942
#CPU times: total: 18min 54s
#Wall time: 1h 22min 38s


best params {'learning_rate': 0.05, 'max_iter': 500}
best score 0.942
CPU times: total: 18min 54s
Wall time: 1h 22min 38s


In [10]:
gs.best_score_

0.942

In [14]:
%%time
#try full dataset on best parameters AdaBoost

###double the estimators
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=9), n_estimators=1000, 
                             learning_rate = 3, random_state=42)

X_train_full = X[:40000]
y_train_full = y[:40000]

X_train_full_proc = preprocess_pipeline.fit_transform(X_train_full)

print('Ada full data', cross_val_score(ada_clf, X_train_full_proc, y_train_full,cv=5,
                                      n_jobs=-1, scoring='accuracy'))

#Ada full data [0.966125 0.95675  0.960625 0.953875 0.962625]
#CPU times: total: 1.33 s
#Wall time: 1h 55min 22s

#average = 0.9599

Ada full data [0.966125 0.95675  0.960625 0.953875 0.962625]
CPU times: total: 1.33 s
Wall time: 1h 55min 22s


In [ ]:
%%time
#Hist grad boost on full dataset


hgb_clf = HistGradientBoostingClassifier(max_depth=9, random_state=42, learning_rate=0.05, max_iter=500)

print('GBC_full', cross_val_score(hgb_clf, X_train_full_proc, y_train_full, 
                                  cv=5, scoring='accuracy', n_jobs=-1))
